<a href="https://colab.research.google.com/github/BRA7534/DeepSeek-Coder/blob/main/DeepSeek_Coder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Charger le modèle directement depuis Hugging Face
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Essayer de charger avec la précision float16 sur GPU
try:
    print("Tentative de chargement sur le GPU avec float16...")
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float16).cuda()
except torch.cuda.OutOfMemoryError:
    print("Mémoire GPU insuffisante. Tentative de chargement sur le CPU avec float16...")
    try:
        # Essayer de charger sur le CPU avec float16
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float16).cpu()
    except RuntimeError:
        print("Mémoire CPU insuffisante pour float16. Chargement avec float32 à la place.")
        # Charger sur le CPU avec float32
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float32).cpu()

def generate_code(prompt):
    # Encoder l'entrée
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Générer une réponse
    outputs = model.generate(**inputs, max_length=512)

    # Libérer la mémoire GPU (si utilisée)
    torch.cuda.empty_cache()

    # Décoder et retourner la réponse
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple d'utilisation
if __name__ == "__main__":
    while True:
        prompt = input("Vous: ")
        if prompt.lower() in ["exit", "quit"]:
            break
        response = generate_code(prompt)
        print("DeepSeek Coder:", response)

Tentative de chargement sur le GPU avec float16...
